In [1]:
import pandas as pd
data = pd.read_csv('training data.csv')

In [11]:
((data.sort_values(by ='max_market_cap_in_eth', ascending=False).max_market_cap_in_eth*3500)/10000000).astype(int)<3931

3       True
65     False
38     False
106    False
15      True
       ...  
115     True
45      True
120     True
58      True
0       True
Name: max_market_cap_in_eth, Length: 125, dtype: bool

In [13]:
data['dollar_marketcap_millions'] = ((data.sort_values(by ='max_market_cap_in_eth', ascending=False).max_market_cap_in_eth*3500)/10000000)

In [31]:
# data = data[data['dollar_marketcap_millions']<300]
data = data[data['max_market_cap_in_eth']<5000]
data

,max_market_cap_in_eth,tfidf_score_per_token_state_variables,tfidf_score_per_token_contract_names,count_vars,number_contracts
0,1,2.235971,1.796620,5,10
1,29,3.728676,2.816017,8,7
2,159,4.895287,2.083885,7,13
4,2,9.213500,2.555208,9,13
5,51,8.873833,2.811664,11,7
...,...,...,...,...,...
119,79,2.446336,1.000000,1,13
120,1,10.444553,3.142687,13,9
121,55,8.233985,3.188274,14,7
122,9,8.592073,2.490608,9,6


In [ ]:
data[['max_market_cap_in_eth', 
                           'tfidf_score_per_token_state_variables', 
                           'tfidf_score_per_token_contract_names', 
                           'count_vars', 
                           'number_contracts']]

In [29]:
3500*1000

3500000

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb

# Assuming 'data' is your DataFrame
# Create a DataFrame from your data array
data = pd.DataFrame(data, columns=['max_market_cap_in_eth', 
                                   'tfidf_score_per_token_state_variables', 
                                   'tfidf_score_per_token_contract_names', 
                                   'count_vars', 
                                   'number_contracts'])

# Create a binary target column
data['binary_target'] = (data['max_market_cap_in_eth'] >= 28).astype(int)

# Splitting the features and new binary target variable
X = data[['tfidf_score_per_token_state_variables', 'tfidf_score_per_token_contract_names', 'count_vars', 'number_contracts']]
y = data['binary_target']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the LightGBM classifier
model = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=100)
model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import lightgbm as lgb

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

[LightGBM] [Info] Number of positive: 61, number of negative: 31
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040231 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65
[LightGBM] [Info] Number of data points in the train set: 92, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.663043 -> initscore=0.676887
[LightGBM] [Info] Start training from score 0.676887
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [40]:
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import lightgbm as lgb
import numpy as np

# Assuming 'data' is your DataFrame
# Create a DataFrame from your data array
data = pd.DataFrame(data, columns=['max_market_cap_in_eth', 
                                   'tfidf_score_per_token_state_variables', 
                                   'tfidf_score_per_token_contract_names', 
                                   'count_vars', 
                                   'number_contracts'])

# Create a binary target column
data['binary_target'] = (data['max_market_cap_in_eth'] >= 28).astype(int)

# Splitting the features and new binary target variable
X = data[['tfidf_score_per_token_state_variables', 'tfidf_score_per_token_contract_names', 'count_vars', 'number_contracts']]
y = data['binary_target']

# Define the model
model = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=100)

# Initialize the cross-validation method
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Create a DataFrame to store the results
results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Perform the 10-fold cross-validation
fold = 0
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    model.fit(X_train, y_train)
    # Predict the test set
    y_pred = model.predict(X_test)

    # Calculate the metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Store the results
    fold_results = pd.DataFrame({
        'Fold': [train_index.size + 1],  # Adjusting fold number to be 1-indexed
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1 Score': [f1]
    })

    # Append current fold results to the cumulative results DataFrame
    results_df = pd.concat([results_df, fold_results], ignore_index=True)

print(results_df)

[LightGBM] [Info] Number of positive: 68, number of negative: 36
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041332 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69
[LightGBM] [Info] Number of data points in the train set: 104, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.653846 -> initscore=0.635989
[LightGBM] [Info] Start training from score 0.635989
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

/var/folders/3d/3_dbb7097rzc81cpv1mrh_nr0000gn/T/ipykernel_28844/3468698578.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, fold_results], ignore_index=True)


[LightGBM] [Info] Number of positive: 68, number of negative: 36
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034761 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 70
[LightGBM] [Info] Number of data points in the train set: 104, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.653846 -> initscore=0.635989
[LightGBM] [Info] Start training from score 0.635989
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

In [49]:
results_df.mean()

Fold            105.4
Accuracy     0.628788
Precision    0.692287
Recall       0.803571
F1 Score     0.737789
dtype: object